In [ ]:
## Imports

# import python libraries  
import numpy as np
import pandas as pd
import sys
import os
import time
import pickle
import tensorflow as tf
import time
from importlib import reload
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from scripts.splitting import split
from scripts.data_wrangling import import_phos, import_SMILES
from scripts.feature_selection import fs_functional_score, fs_atlas_landmark, true_phosphosite_filter
from scripts.data_preparation import data_prep_SMILES, encode_SMILES
from scripts.data_selection import data_indexing
from scripts.mean_model import MeanModel
from scripts.ML_DRP_model import train_model_SMILES, train_model_SMILES_cv, prep_xd, prediction_metrics
tf.keras.utils.set_random_seed(42) # set random seeds for Python, NumPy, and TensorFlow

print('Imports: Done')



## Data Wrangling

# Import and wrangle phosphoproteomics and GDSC1 drug data
phos_df, drug_df, drug_matrix, _all_cls, _all_drugs, common_ind  = import_phos()
SMILES_df = import_SMILES()
## Feature Selection 

# create list of phosphosites filtered by feature selection method
#phosphosites = fs_functional_score(phos_df,95)
phosphosites = fs_atlas_landmark(phos_df,80)

# remove false positive phosphosites
filtered_phosphosites = true_phosphosite_filter(phosphosites)

# reduce remaining features down by feature selection
phos_df = phos_df.filter(filtered_phosphosites,axis=1)

## Data Preparation & Selection 
one_hot_SMILES = encode_SMILES(SMILES_df) # one-hot encode SMILES strings
x_drug, x_all, y_series = data_prep_SMILES(drug_df,one_hot_SMILES,phos_df,common_ind)
drug_cl_pairs =  y_series.index

# convert one-hot encoded SMILES into an array of values, used for model input shape
xd_vals = prep_xd(x_drug)

# Cancer blind train-test split
rand_seed = 42 # set random seed for train-test split
train_size = 0.75 # set train-test fraction

train_pairs, test_pairs = split(rand_seed, _all_cls, _all_drugs, drug_cl_pairs, 
                                train_size, split_type='cblind')

# index for each set of train-test pairs
xo_train, xd_train, y_train, xo_test, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all,x_drug,y_series) # select train and test


## Neural Network 

In [ ]:
## Model Building

# Learning rate schedule function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


## CNN using functional API
def build_model(learning_rate=1e-3, momentum=0, seed=42):

    # set weight initialiser
    initializer = tf.keras.initializers.GlorotUniform(seed)
    
    # omics data input
    x_input = layers.Input(shape=(xo_train.shape[1],1))
    # 1st convolution layer
    x = layers.Conv1D(filters=8, kernel_size=8, kernel_initializer=initializer, activation='relu')(x_input) 
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1)(x)
    # 2nd convolution layer
    x = layers.Conv1D(filters=16, kernel_size=8, kernel_initializer=initializer, activation='relu')(x) 
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Flatten()(x) 

    # FC layer for xo_train
    x = layers.Dense(256, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    
    # one-hot encoded drug data input
    y_input = layers.Input(shape=(xd_vals.shape[1:]))

    # 1st convolution layer
    y = layers.Conv1D(filters=256, kernel_size=11, kernel_initializer=initializer, activation='relu')(y_input) 
    y = layers.BatchNormalization()(y)
    y = layers.MaxPooling1D(pool_size=3, strides=3)(y)
    y = layers.Dropout(0.1)(y)
    y = layers.Flatten()(y) 
    
    # FC layer for xd_train
    y = layers.Dense(64, kernel_initializer=initializer, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dense(64, kernel_initializer=initializer, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.5)(y)
    
    
    # Concatenate omics and encoded drug data
    z = layers.concatenate([x, y])
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(1, kernel_initializer=initializer)(z)

    model = tf.keras.Model([x_input, y_input], z)
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate, 
                                                        momentum=momentum), 
                                                        loss='mse', metrics=['mae'])
    return model

print('Model Building: Done')

In [ ]:
# train model without cross-validation

m_func = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler
epochs = 100

train_model_SMILES(m_func,lr_scheduler, 
          x_train = [xo_train, xd_train], x_test = [xo_test, xd_test], 
          y_train = y_train, y_test = y_test, 
          epochs = epochs)

In [ ]:
# train model with cross-validation

epochs = 100
rand_seed_list = [913,425]
model = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler

train_model_SMILES_cv(model,lr_scheduler,
                      train_pairs,x_all,x_drug,y_series,
                      epochs,rand_seed_list)